In [ ]:
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import math
from sklearn.model_selection import StratifiedKFold

# (Precision/Recall) in All W2V

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for k in range(len(W2V)):
    # model load
    model = Word2Vec.load(W2V[k])
    word_vectors = model.wv
    vocabs = word_vectors.key_to_index

    # average embedding
    zeros = np.zeros((int(W2V[k][10:12]),))
    document_embedding_list, commentnum = get_document_vectors(gf['Contents'])
    
    # DataFrame
    gf['embedding'] = document_embedding_list
    new_gf = gf[['Post','Replies','Views','Comment#','Len', 'Days', 'Link', "visit", "read", "topic", "posts","given", "received", "topic_c", "post_c",'embedding','P','Q']]
    test_gf = new_gf[:1097]
    
    # X,y
    X, y = data_split(test_gf)
    
    res = pd.DataFrame({'W2V':[0, 0, 0, 0, 0, 0]}, index = ['Acc','PRE','REC','F1','ROCAUC','BEST'])
    res.iloc[0,0] = "({0},{1},{2},{3})_10CV".format(W2V[k][10:12],W2V[k][13],W2V[k][15],W2V[k][17])
    res.iloc[1,0] = k
        
    # Data for MLP
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    print(y.value_counts())  

    U = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 
         0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39,
         0.40, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.50]

    # for lr in LR:
    for unit in U:
#         res.iloc[0,0] = unit
        y_predown = [1.00 if x==1.0 else x for x in yp]
        y_predown = [1.00 if x==0.666666667 else x for x in y_predown]
        y_predown = [0+unit if x==0.0 else x for x in y_predown]
        y_predown = [0+unit if x==0.333333333 else x for x in y_predown]
        y_predown = pd.DataFrame(y_predown)
        y_recdown = [1-unit if x==1.0 else x for x in yp]
        y_recdown = [1-unit if x==0.666666667 else x for x in y_recdown]
        y_recdown = [0.00 if x==0.0 else x for x in y_recdown]
        y_recdown = [0.00 if x==0.333333333 else x for x in y_recdown]
        y_recdown = pd.DataFrame(y_recdown)       

        #===============================================================================================================    
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 0)
        n_iter = 0

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        bce010_acc = []
        bce010_pre = []
        bce010_rec = []
        bce010_f1 = []
        bce010_rocauc = []

        model_005 = mlp_model()
        model_010 = mlp_model()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)

        for train_index, test_index in skf.split(X, y):
            #################
            set_seed(seed=2)
            #################
            n_iter += 1
            X_train = X.iloc[train_index]
            y_train= y.iloc[train_index]
            y_predown_train= y_predown.iloc[train_index]
            y_recdown_train= y_recdown.iloc[train_index]
            X_test = X.iloc[test_index]
            y_test= y.iloc[test_index]
            print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            y_train = y_train.astype(float)
            y_predown_train = np.array(y_predown_train)
            y_predown_train = y_predown_train.astype(float)
            y_recdown_train = np.array(y_recdown_train)
            y_recdown_train = y_recdown_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)

            # MLP_BCE(y_005)
            print('MLP_BCE(y_005)')
            model_005.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = learning_rate), metrics=['accuracy'])
            history = model_005.fit(X_train, y_predown_train, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch_size, callbacks=[early_stopping])
        #     plt.plot(history.history['loss'], label='loss')
        #     plt.ylim([0, 1])
        #     plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
        #     plt.ylabel('Loss',fontweight="bold",fontsize = 15)
        #     plt.title("Cost Function",fontweight="bold",fontsize = 20)
        #     plt.legend()
        #     plt.show()
            predicted = np.round(model_005.predict(X_test))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))

            # MLP_BCE(y_010)
            print('MLP_BCE(y_010)')
            model_010.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = learning_rate), metrics=['accuracy'])
            history = model_010.fit(X_train, y_recdown_train, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch_size, callbacks=[early_stopping])
        #     plt.plot(history.history['loss'], label='loss')
        #     plt.ylim([0, 1])
        #     plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
        #     plt.ylabel('Loss',fontweight="bold",fontsize = 15)
        #     plt.title("Cost Function",fontweight="bold",fontsize = 20)
        #     plt.legend()
        #     plt.show()
            predicted = np.round(model_010.predict(X_test))
            bce010_acc.append(metrics.accuracy_score(y_test, predicted))
            bce010_pre.append(metrics.precision_score(y_test, predicted))
            bce010_rec.append(metrics.recall_score(y_test, predicted))
            bce010_f1.append(metrics.f1_score(y_test, predicted))
            bce010_rocauc.append(metrics.roc_auc_score(y_test, predicted))

        res['MLP(y_predown)_{}'.format(unit)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc), '']
        res['MLP(y_recdown)_{}'.format(unit)] = [np.mean(bce010_acc), np.mean(bce010_pre), np.mean(bce010_rec), np.mean(bce010_f1), np.mean(bce010_rocauc), '']

    res.to_csv("Helpful_(Pre_Rec_experiment).csv", mode = 'a', float_format='%.4g')